# Training Code
This workbook is just for training the yolov9 and the detr models. In the legacy folder there is an old workbook which indicates how the dataset was scraped from the internet and then put into a standard Yolo format. 

In [1]:
# !{os.sys.executable} -m pip install --upgrade ultralytics==8.1.29


In [2]:
# import packages
import os
import cv2
import numpy as np
from IPython.display import Video, display
import time
import torch


from utils.pretrained_deployment import download_images, download_images2, download_images_with_resize, download_images_full_size
from utils.display import *
from utils.make_dataset_nyc_landmarks import make_nyc_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device", device)

# reload modules every 2 seconds to see changes in notebook
%load_ext autoreload
%autoreload 2

%matplotlib inline

device cuda


In [3]:
classes = {
    0: "EmpireState",
    1: "WTC",
    2: "432ParkAve",
    3: "UNBuilding",
    4: "Flatiron",
    5: "BrooklynBridge",
    6: "ChryslerBuilding",
    7: "MetlifeBuilding",
    8: "StatueOfLiberty",
    9: "30HudsonYards",
}

## Training Yolov9

In [4]:
from ultralytics import YOLO

# models to experiment with 
# YOLOv8l
# YOLOv8n
# yolov9c.pt

# Build a YOLOv9c model from pretrained weight
# pretrained_weights = 'yolov8l.pt'
pretrained_weights = 'yolov8n.pt'
model = YOLO(pretrained_weights)

# Display model information (optional)
model.info()

dataset_yaml_path = '/home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/dataset.yaml' # vm
# dataset_yaml_path = '/Users/johansweldens/Documents/EECS6692.DLoE/final_project/e6692-2024spring-finalproject-jwss-jws2215/dataset.yaml' # local

# Train model
# device = "mps"
results = model.train(data= dataset_yaml_path, epochs=50, imgsz=640, device = device)

YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs
New https://pypi.org/project/ultralytics/8.2.4 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/train.cache... 944 images, 0 backgrounds, 0 corrupt: 100%|██████████| 944/944 [00:00<?, 
val: Scanning /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/valid.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?i


Plotting labels to runs/detect/train12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/59 [00:00<?, ?it/s]Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22queryClusterPropertiesERPhS3_, version libcudnn_cnn_infer.so.8
Could not load library libcudnn_cnn_train.so.8. Error: /usr/local/cuda/lib64/libcudnn_cnn_train.so.8: undefined symbol: _ZN5cudnn3cnn5infer22query

RuntimeError: GET was unable to find an engine to execute this computation

## Training RT-DETR-L

In [3]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')

# Display model information (optional)
model.info()

# Train model
results = model.train(data='/home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/dataset.yaml', epochs=50, imgsz=640)

rt-detr-l summary: 673 layers, 32970476 parameters, 0 gradients, 108.3 GFLOPs
Ultralytics YOLOv8.2.0 🚀 Python-3.10.13 torch-2.1.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=/home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

train: Scanning /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/train.cache... 944 images, 0 backgrounds, 0 corrupt: 100%|██████████| 944/944 [00:00<?, 
val: Scanning /home/jws2215/e6692-2024spring-finalproject-jwss-jws2215/datasets/nyc_landmarks/valid.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?i


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 143 weight(decay=0.0), 206 weight(decay=0.0005), 226 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/59 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
       1/50      13.4G      1.017      28.84     0.9115         47        640: 100%|██████████| 59/59 [00:31<00:00,  1.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.38it/s]

                   all        300        381       0.32     0.0709     0.0402     0.0282


...

  0%|          | 0/59 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
      50/50      13.6G     0.1024     0.1655    0.06901         20        640: 100%|██████████| 59/59 [00:28<00:00,  2.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.83it/s]

                   all        300        381      0.969      0.939      0.955      0.859



50 epochs completed in 0.479 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 66.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 66.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.0 🚀 Python-3.10.13 torch-2.1.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
rt-detr-l summary: 498 layers, 32004290 parameters, 0 gradients, 103.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.33it/s]


                   all        300        381      0.979       0.93       0.95      0.862
           EmpireState        300         53      0.962      0.955      0.935      0.774
                   WTC        300         61      0.976          1      0.987        0.7
            432ParkAve        300         22          1      0.907      0.956       0.83
            UNBuilding        300         34          1      0.963      0.994      0.975
              Flatiron        300         29      0.964          1      0.995      0.956
        BrooklynBridge        300         28      0.978      0.964      0.965      0.902
      ChryslerBuilding        300         60      0.976      0.917      0.958      0.905
       MetlifeBuilding        300         27          1      0.871      0.912      0.871
       StatueOfLiberty        300         49          1      0.959      0.968      0.929
         30HudsonYards        300         18      0.932      0.766       0.83      0.777
Speed: 0.1ms preproce